 * 라이브러리 호출

In [1]:
import numpy as np
import pandas as pd
import os
from collections import Counter
import datetime
from random import *
import re
import requests
from tqdm import tqdm


import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
from matplotlib.font_manager import fontManager
import plotly as py
import plotly.express as px
from pprint import pprint

import lxml
import bs4
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import seaborn as sns
import cufflinks as cf
%matplotlib notebook
cf.go_offline()


 * 폰트

In [2]:
for font in fontManager.ttflist:
    if 'Mal' in font.name:
        print(font.name)
plt.rc('font', family='Malgun Gothic')

Lohit Malayalam
Samyak Malayalam
RaghuMalayalam


 * 데이터 파일 불러오기

In [98]:
# col_dtype = {'act_kwd':'str'}

In [3]:
merge_table = pd.read_csv('./merge_raw_1234.csv')
# merge_table = pd.read_csv('./merge_raw_1234.csv', dtype=col_dtype)
merge_table.drop(['Unnamed: 0'], axis=1)

/home/aiffel0042/anaconda3/envs/aiffel/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning:

Columns (17,27) have mixed types.Specify dtype option on import or set low_memory=False.



,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,...,de_dt,de_tm,buy_am,buy_ct,clnt_gender,clnt_age,clac_nm1,clac_nm2,clac_nm3,pd_c_2
0,72429,1,2,2,A03,20190919,22:09,842130,NaN,NaN,...,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN
1,72429,1,1,1,A03,20190919,22:09,839064,NaN,NaN,...,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN
2,72428,5,1,0,A01,20190918,06:37,12568,NaN,라인에디션블라우스,...,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN
3,72428,4,1,0,A01,20190914,23:42,243555,NaN,펜디가방,...,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN
4,72428,3,3,0,A01,20190914,21:50,839771,NaN,페레가모가방,...,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3196357,1,2,2,0,A01,20190922,14:10,56113,NaN,초등가을점퍼,...,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN
3196358,1,2,1,0,A01,20190922,14:09,41584,NaN,초등가을잠바,...,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN
3196359,1,1,3,0,A01,20190911,16:15,36140,NaN,과일바구니,...,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN
3196360,1,1,2,0,A01,20190911,16:15,22432,NaN,과일선물세트 백화점,...,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN


In [4]:
merge_table.head()

,Unnamed: 0,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,...,de_dt,de_tm,buy_am,buy_ct,clnt_gender,clnt_age,clac_nm1,clac_nm2,clac_nm3,pd_c_2
0,0,72429,1,2,2,A03,20190919,22:09,842130,NaN,...,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN
1,1,72429,1,1,1,A03,20190919,22:09,839064,NaN,...,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN
2,2,72428,5,1,0,A01,20190918,06:37,12568,NaN,...,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN
3,3,72428,4,1,0,A01,20190914,23:42,243555,NaN,...,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN
4,4,72428,3,3,0,A01,20190914,21:50,839771,NaN,...,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN


* 키워드 리스트 불러오기

In [5]:
act_kwd = list(merge_table['act_kwd'].unique())
act_kwd.remove(np.nan)
act_kwd

['라인에디션블라우스',
 '펜디가방',
 '페레가모가방',
 '루이비통',
 '구찌',
 '속옷세트',
 '닥스크로스백남성',
 'hd9743',
 '핫도그',
 '원피스',
 '아이패스파워',
 '계란',
 'gotqks',
 'enqn',
 '멜빵바지',
 '에어박스 280',
 '세젤예 피크닉매트',
 '세젤예',
 'LG통돌이세탁기',
 '보쌈용',
 '삼겹',
 'tkaruq',
 '사겹',
 '보쌈',
 '수박',
 '앞다리',
 '찌개용',
 '비비고포기김치',
 '미숫가루',
 '햇반210g',
 '계란30',
 '종근당 락토핏 코어',
 '종근당 락토핏',
 '비엔나',
 '콩장조림',
 '팸퍼스',
 '호두',
 'dㅜ유',
 '닭갈비',
 '피쇼',
 '펩시',
 '골뱅이',
 '캐논 m50',
 '쌀',
 '맥스클리닉',
 '빠질라카노',
 '브라운 맨투맨',
 '브라운 맨투맴',
 '썬밸리 워터파크',
 '구찌 지갑',
 '남자미니클립',
 '남자구찌지갑',
 '프라다 지갑',
 '빈폴지갑',
 '프라다지갑',
 '프라다남성지갑',
 '프라다지갑구찌지갑',
 '프라다지갑 구찌지갑',
 '남자지갑',
 '프라다지가',
 '몽블랑지갑',
 '지갑',
 '토마토쿨리',
 '베이퍼맥스',
 '떡',
 '비누',
 '시리아비누',
 '천연비누',
 '밍크코트',
 '가죽잠바',
 '가죽징바',
 '고추장',
 '최유라쇼',
 '닥스양산',
 '양산',
 '양산 uv차단',
 '키친타올',
 '키칫',
 '서큘레이터',
 '파세코',
 '르젠',
 '리리코스',
 '리리코스마린콜라겐쿠션',
 '파세코 써큘레이터',
 '고시히카리',
 '잘모이',
 '천년의신비보이차',
 '파타고니아후리스',
 '파타고니아 클레식 레트로엑스 후리스 플리스 자켓',
 '물걸레청소포',
 '물걸레청소기 일회용패드',
 '물걸레 일회용패드',
 '써스데이아일랜드원피스',
 '만두피',
 '생만두피',
 '만두피.',
 '냉동만두피',
 '왕만두피',
 '한경희죽제조기'

In [107]:
act_kwd[0]

'라인에디션블라우스'

In [108]:
type(act_kwd[0])

str

In [109]:
len(act_kwd)

101952

 * 시범 삼아 1개 추출

In [112]:
# Selenium이 사용할 브라우저 선택 및 크롬 드라이버 실행
driver=webdriver.Chrome('./chromedriver')

In [78]:
# 웹 페이지를 읽어오기
driver.get('http://www.lotteimall.com/main/viewMain.lotte?dpml_no=1&tlog=00100_1')

# find_element_by_id : id 속성으로 요소 하나 추출

a_element1=driver.find_element_by_id('headerQuery') # headerQuery : 검색어 입력 창
a_element1.click()                                  # 해당 요소를 클릭
a_element1.send_keys('로가디스')                     # 키를 입력


a_element2=driver.find_element_by_id("btn_headerSearch") # btn_headerSearch : 검색 버튼
a_element2.click()

a_element3=driver.find_element_by_class_name("content_category")

In [79]:
a_element3.text

'패션의류/언더웨어\n남성정장/셔츠 (1,133)\n남성캐주얼 (822)\n진/유니섹스 (52)\n명품/쥬얼리\n시계/보석/쥬얼리 (221)\n스포츠/레저\n스포츠패션/슈즈 (37)\n영플라자\n진/유니섹스 (26)\n롯데백화점\n진/유니섹스 (12)\n남성정장/셔츠 (3)\n남성캐주얼 (1)'

In [80]:
a_element4=a_element3.text.split('\n')

In [81]:
a_element4

['패션의류/언더웨어',
 '남성정장/셔츠 (1,133)',
 '남성캐주얼 (822)',
 '진/유니섹스 (52)',
 '명품/쥬얼리',
 '시계/보석/쥬얼리 (221)',
 '스포츠/레저',
 '스포츠패션/슈즈 (37)',
 '영플라자',
 '진/유니섹스 (26)',
 '롯데백화점',
 '진/유니섹스 (12)',
 '남성정장/셔츠 (3)',
 '남성캐주얼 (1)']

In [82]:
test=[]
for i in range(len(a_element4)):
    text = re.sub(r'\([^)]*\)', repl='', string=a_element4[i])
    text=text.rstrip()
    test.append(text)
a_element5=pd.Series(test,name=1)

In [83]:
a_element5

0     패션의류/언더웨어
1       남성정장/셔츠
2         남성캐주얼
3        진/유니섹스
4        명품/쥬얼리
5     시계/보석/쥬얼리
6        스포츠/레저
7      스포츠패션/슈즈
8          영플라자
9        진/유니섹스
10        롯데백화점
11       진/유니섹스
12      남성정장/셔츠
13        남성캐주얼
Name: 1, dtype: object

In [122]:
act_kwd_cat=pd.Series(['시작'],name='start')
act_kwd_cat

0    시작
Name: start, dtype: object

In [117]:
driver=webdriver.Chrome('./chromedriver')  # Selenium이 사용할 브라우저 선택 및 크롬 드라이버 실행

In [118]:
# # 20개만 먼저
# # 행 추가 방식
# for i in tqdm(range(20)):
    
#     driver.implicitly_wait(randint(2,6))
#     driver.get('http://www.lotteimall.com/main/viewMain.lotte?dpml_no=1&tlog=00100_1')
#     a_element1=driver.find_element_by_id('headerQuery')
#     a_element1.click()
#     a_element1.send_keys(act_kwd[i])
#     a_element2=driver.find_element_by_id("btn_headerSearch")
#     a_element2.click()
#     try:
#         if ('결과' in driver.find_element_by_class_name('center').text) == True:

#             a_element3=driver.find_element_by_class_name("content_category")
#             a_element4=a_element3.text.split('\n')
#             test=[]
#             for j in range(len(a_element4)):
#                 text = re.sub(r'\([^)]*\)', repl='', string=a_element4[j])
#                 text=text.rstrip()
#                 test.append(text)
#             a_element5=pd.Series(test[0],name=i)
#             act_kwd_cat=pd.concat([act_kwd_cat,a_element5],axis=0)
#         else:
#             a_element6=pd.Series(['검색이 되지 않는다'],name=i)
#             act_kwd_cat=pd.concat([act_kwd_cat,a_element6],axis=0)
#     except:
#         a_element6=pd.Series(['아예 검색 없음'],name=i)
#         act_kwd_cat=pd.concat([act_kwd_cat,a_element6],axis=0)

100%|██████████| 20/20 [01:01<00:00,  3.09s/it]


In [123]:
# 5개만 먼저
# 열 추가 방식
for i in tqdm(range(5)):
    
    driver.implicitly_wait(randint(2,6))
    driver.get('http://www.lotteimall.com/main/viewMain.lotte?dpml_no=1&tlog=00100_1')
    a_element1=driver.find_element_by_id('headerQuery')
    a_element1.click()
    a_element1.send_keys(act_kwd[i])
    a_element2=driver.find_element_by_id("btn_headerSearch")
    a_element2.click()
    try:
        if ('결과' in driver.find_element_by_class_name('center').text) == True:

            a_element3=driver.find_element_by_class_name("content_category")
            a_element4=a_element3.text.split('\n')
            test=[]
            for j in range(len(a_element4)):
                text = re.sub(r'\([^)]*\)', repl='', string=a_element4[j])
                text=text.rstrip()
                test.append(text)
            a_element5=pd.Series(test,name=i)
            act_kwd_cat=pd.concat([act_kwd_cat,a_element5],axis=1)
        else:
            a_element6=pd.Series(['검색이 되지 않는다'],name=i)
            act_kwd_cat=pd.concat([act_kwd_cat,a_element6],axis=1)
    except:
        a_element6=pd.Series(['아예 검색 없음'],name=i)
        act_kwd_cat=pd.concat([act_kwd_cat,a_element6],axis=1)

100%|██████████| 5/5 [00:14<00:00,  2.98s/it]


In [125]:
act_kwd[0:4]

['라인에디션블라우스', '펜디가방', '페레가모가방', '루이비통']

In [124]:
act_kwd_cat

,start,0,1,2,3,4
0,시작,패션의류/언더웨어,명품/쥬얼리,명품/쥬얼리,명품/쥬얼리,명품/쥬얼리
1,NaN,여성트랜드의류,수입명품/해외배송,수입명품/해외배송,수입명품/해외배송,수입명품/해외배송
2,NaN,여성브랜드의류,스포츠/레저,스포츠/레저,주방/생활/건강,시계/보석/쥬얼리
3,NaN,롯데백화점,스포츠패션/슈즈,스포츠패션/슈즈,문구/악기/도서/취미,패션의류/언더웨어
4,NaN,여성캐주얼,롯데백화점,잡화/슈즈,세제/욕실/위생,여성브랜드의류
5,NaN,영캐주얼,패션잡화,패션잡화,U아동관,여성트랜드의류
6,NaN,NaN,잡화/슈즈,롯데백화점,유아동도서/완구/교구,진/유니섹스
7,NaN,NaN,패션잡화,패션잡화,가전/디지털,남성캐주얼
8,NaN,NaN,NaN,여성정장/모피,휴대폰/음향/차량용기기,남성정장/셔츠
9,NaN,NaN,NaN,NaN,NaN,잡화/슈즈


In [126]:
# 파일 저장
act_kwd_cat.to_csv("ex1.csv", mode='w')

In [6]:
act_kwd_cat=pd.Series(['시작'],name='start')
act_kwd_cat

0    시작
Name: start, dtype: object

In [7]:
driver=webdriver.Chrome('./chromedriver')  # Selenium이 사용할 브라우저 선택 및 크롬 드라이버 실행

In [8]:
# 1000개만 먼저
# 열 추가 방식
for i in tqdm(range(1000)):
    
    driver.implicitly_wait(randint(2,6))
    driver.get('http://www.lotteimall.com/main/viewMain.lotte?dpml_no=1&tlog=00100_1')
    a_element1=driver.find_element_by_id('headerQuery')
    a_element1.click()
    a_element1.send_keys(act_kwd[i])
    a_element2=driver.find_element_by_id("btn_headerSearch")
    a_element2.click()
    try:
        if ('결과' in driver.find_element_by_class_name('center').text) == True:

            a_element3=driver.find_element_by_class_name("content_category")
            a_element4=a_element3.text.split('\n')
            test=[]
            for j in range(len(a_element4)):
                text = re.sub(r'\([^)]*\)', repl='', string=a_element4[j])
                text=text.rstrip()
                test.append(text)
            a_element5=pd.Series(test,name=i)
            act_kwd_cat=pd.concat([act_kwd_cat,a_element5],axis=1)
        else:
            a_element6=pd.Series(['검색이 되지 않는다'],name=i)
            act_kwd_cat=pd.concat([act_kwd_cat,a_element6],axis=1)
    except:
        a_element6=pd.Series(['아예 검색 없음'],name=i)
        act_kwd_cat=pd.concat([act_kwd_cat,a_element6],axis=1)

100%|██████████| 1000/1000 [51:40<00:00,  3.10s/it] 


In [9]:
# 파일 저장
act_kwd_cat.to_csv("sample_1000.csv", mode='w')

In [10]:
# 1000~2000개
# 열 추가 방식
for i in tqdm(range(1000, 2000)):
    
    driver.implicitly_wait(randint(2,6))
    driver.get('http://www.lotteimall.com/main/viewMain.lotte?dpml_no=1&tlog=00100_1')
    a_element1=driver.find_element_by_id('headerQuery')
    a_element1.click()
    a_element1.send_keys(act_kwd[i])
    a_element2=driver.find_element_by_id("btn_headerSearch")
    a_element2.click()
    try:
        if ('결과' in driver.find_element_by_class_name('center').text) == True:

            a_element3=driver.find_element_by_class_name("content_category")
            a_element4=a_element3.text.split('\n')
            test=[]
            for j in range(len(a_element4)):
                text = re.sub(r'\([^)]*\)', repl='', string=a_element4[j])
                text=text.rstrip()
                test.append(text)
            a_element5=pd.Series(test,name=i)
            act_kwd_cat=pd.concat([act_kwd_cat,a_element5],axis=1)
        else:
            a_element6=pd.Series(['검색이 되지 않는다'],name=i)
            act_kwd_cat=pd.concat([act_kwd_cat,a_element6],axis=1)
    except:
        a_element6=pd.Series(['아예 검색 없음'],name=i)
        act_kwd_cat=pd.concat([act_kwd_cat,a_element6],axis=1)

100%|██████████| 1000/1000 [56:19<00:00,  3.38s/it] 


In [11]:
# 파일 저장
act_kwd_cat.to_csv("sample_2000.csv", mode='w')

In [12]:
# 2000~3000개
# 열 추가 방식
for i in tqdm(range(2000, 3000)):
    
    driver.implicitly_wait(randint(2,6))
    driver.get('http://www.lotteimall.com/main/viewMain.lotte?dpml_no=1&tlog=00100_1')
    a_element1=driver.find_element_by_id('headerQuery')
    a_element1.click()
    a_element1.send_keys(act_kwd[i])
    a_element2=driver.find_element_by_id("btn_headerSearch")
    a_element2.click()
    try:
        if ('결과' in driver.find_element_by_class_name('center').text) == True:

            a_element3=driver.find_element_by_class_name("content_category")
            a_element4=a_element3.text.split('\n')
            test=[]
            for j in range(len(a_element4)):
                text = re.sub(r'\([^)]*\)', repl='', string=a_element4[j])
                text=text.rstrip()
                test.append(text)
            a_element5=pd.Series(test,name=i)
            act_kwd_cat=pd.concat([act_kwd_cat,a_element5],axis=1)
        else:
            a_element6=pd.Series(['검색이 되지 않는다'],name=i)
            act_kwd_cat=pd.concat([act_kwd_cat,a_element6],axis=1)
    except:
        a_element6=pd.Series(['아예 검색 없음'],name=i)
        act_kwd_cat=pd.concat([act_kwd_cat,a_element6],axis=1)

100%|██████████| 1000/1000 [55:41<00:00,  3.34s/it] 


In [ ]:
# 파일 저장
act_kwd_cat.to_csv("sample_2000.csv", mode='w')

In [132]:
# 5000개만 먼저
# 열 추가 방식
for i in tqdm(range(5000)):
    
    driver.implicitly_wait(randint(2,6))
    driver.get('http://www.lotteimall.com/main/viewMain.lotte?dpml_no=1&tlog=00100_1')
    a_element1=driver.find_element_by_id('headerQuery')
    a_element1.click()
    a_element1.send_keys(act_kwd[i])
    a_element2=driver.find_element_by_id("btn_headerSearch")
    a_element2.click()
    try:
        if ('결과' in driver.find_element_by_class_name('center').text) == True:

            a_element3=driver.find_element_by_class_name("content_category")
            a_element4=a_element3.text.split('\n')
            test=[]
            for j in range(len(a_element4)):
                text = re.sub(r'\([^)]*\)', repl='', string=a_element4[j])
                text=text.rstrip()
                test.append(text)
            a_element5=pd.Series(test,name=i)
            act_kwd_cat=pd.concat([act_kwd_cat,a_element5],axis=1)
        else:
            a_element6=pd.Series(['검색이 되지 않는다'],name=i)
            act_kwd_cat=pd.concat([act_kwd_cat,a_element6],axis=1)
    except:
        a_element6=pd.Series(['아예 검색 없음'],name=i)
        act_kwd_cat=pd.concat([act_kwd_cat,a_element6],axis=1)

 11%|█         | 540/5000 [1:56:13<15:59:56, 12.91s/it]


TimeoutException: Message: timeout: Timed out receiving message from renderer: 299.284
  (Session info: chrome=87.0.4280.66)


In [ ]:
# 파일 저장
act_kwd_cat.to_csv("sample_5000.csv", mode='w')

In [90]:
for i in tqdm(range(len(act_kwd))):
    
    driver.implicitly_wait(randint(2,6))
    driver.get('http://www.lotteimall.com/main/viewMain.lotte?dpml_no=1&tlog=00100_1')
    a_element1=driver.find_element_by_id('headerQuery')
    a_element1.click()
    a_element1.send_keys(act_kwd[i])
    a_element2=driver.find_element_by_id("btn_headerSearch")
    a_element2.click()
    try:
        if ('결과' in driver.find_element_by_class_name('center').text) == True:

            a_element3=driver.find_element_by_class_name("content_category")
            a_element4=a_element3.text.split('\n')
            test=[]
            for j in range(len(a_element4)):
                text = re.sub(r'\([^)]*\)', repl='', string=a_element4[j])
                text=text.rstrip()
                test.append(text)
            a_element5=pd.Series(test[0],name=i)
            act_kwd_cat=pd.concat([act_kwd_cat,a_element5],axis=1)
        else:
            a_element6=pd.Series(['검색이 되지 않는다'],name=i)
            act_kwd_cat=pd.concat([act_kwd_cat,a_element6],axis=1)
    except:
        a_element6=pd.Series(['아예 검색 없음'],name=i)
        act_kwd_cat=pd.concat([act_kwd_cat,a_element6],axis=1)

  0%|          | 0/101953 [00:00<?, ?it/s]


WebDriverException: Message: chrome not reachable
  (Session info: chrome=87.0.4280.66)


In [43]:
a02_brand

,start,0,1,2,3,4,5,6,7,8,...,4830,4831,4832,4833,4834,4835,4836,4837,4838,4839
0,시작,로가디스,아이슈즈,아이슈즈,신일,스파클,헤지스키즈,실리트,센스맘,빈폴스포츠,...,세컨스킨,아디다스,닥스_핸드백,스킨알엑스,마루엔개비,빈폴ACC,빈폴ACC,에이치플러스몰,타미힐피거 여성,SOUP
1,NaN,로가디스 시계,고세,멜팅브라운,신일산업,한모금,헤지스ACC,1300K,NaN,시드마켓,...,엠피지,NaN,질스튜어트_핸드백,롭스,한샘,빈폴스포츠,빈폴 레이디스,버버리,타미힐피거여성,후라밍고
2,NaN,NaN,멜팅브라운,텐바이텐,신일산업,기타,모노블랙,바보사랑,NaN,나야홍,...,톰앤래빗,NaN,헤지스 가방,네이처 리퍼블릭,아씨방,빈폴-골프,NaN,프라다,타미힐피거,쇼크업소버코리아
3,NaN,NaN,크록스,에이치플러스몰,신일산업,네슬레,레노마키즈,아트박스,NaN,헤지스골프,...,아디다스,NaN,바보사랑,눙크,이케아,헤지스골프,NaN,발렌시아가,타미힐피거,마쉬옐로우
4,NaN,NaN,소다,디자이어로즈,NaN,롯데칠성음료,엠제이 주니어,실리트_,NaN,아디다스,...,씨,NaN,후추통,온더바디,지비엠,NaN,NaN,발렌티노,타미힐피거,에이치스타일
5,NaN,NaN,미소페,아트박스,NaN,미스테리월,NaN,룸멜스바하,NaN,반에이크 티알엔티,...,베스띠벨리,NaN,엘케이트,이니스프리,시니프,NaN,NaN,구찌,폴로,퍼스웰
6,NaN,NaN,베리슈,비비핑크,NaN,NaN,NaN,NaN,NaN,와이드앵글,...,베스띠벨리,NaN,빌리버스*,네이처리퍼블릭,더리체,NaN,NaN,생로랑,타미힐피거진,닉스
7,NaN,NaN,텐바이텐,1300K,NaN,NaN,NaN,NaN,NaN,막스마라,...,파파야,NaN,빈폴ACC,해피바스,마켓리더,NaN,NaN,돌체앤가바나,타미힐피거 슈즈,에이치플러스몰
8,NaN,NaN,빌리버스,후추통,NaN,NaN,NaN,NaN,NaN,아이더,...,비꼴리끄,NaN,기타,엘지생활건강,라자가구,NaN,NaN,지방시,타미힐피거 남성,사토리
9,NaN,NaN,게스슈즈,버켄스탁,NaN,NaN,NaN,NaN,NaN,네파,...,아디다스,NaN,웅가로_넥타이,푸드어홀릭,나무뜰,NaN,NaN,레스포색,NaN,모슬린


In [42]:
a02_brand.to_csv('a02_brand_ing.csv')

In [ ]:
#4839

In [51]:
a02_brand.drop(a02_brand.iloc[:,[-5,-4,-3,-2,-1]],axis=1,inplace=True)

In [52]:
a02_kwd2=a02_kwd[4840:]

In [54]:
a02_kwd2[0]

'머렐여성패딩'

In [53]:
a02_brand2=pd.Series([1],name='start')

In [56]:
for i in range(len(a02_kwd2)):
    
    driver.implicitly_wait(randint(2,6))
    driver.get('http://www.lotteimall.com/main/viewMain.lotte?dpml_no=1&tlog=00100_1')
    a_element1=driver.find_element_by_id('headerQuery')
    a_element1.click()
    a_element1.send_keys(a02_kwd2[i])
    a_element2=driver.find_element_by_id("btn_headerSearch")
    a_element2.click()
    try:
        if ('결과' in driver.find_element_by_class_name('center').text) == True:

            a_element3=driver.find_element_by_id("brndList")
            a_element4=a_element3.text.split('\n')
            test=[]
            for j in range(len(a_element4)):
                text = re.sub(r'\([^)]*\)', repl='', string=a_element4[j])
                text=text.rstrip()
                test.append(text)
            a_element5=pd.Series(test,name=i)
            a02_brand2=pd.concat([a02_brand2,a_element5],axis=1)
        else:
            a_element6=pd.Series(['검색이 되지 않는다'],name=i)
            a02_brand2=pd.concat([a02_brand2,a_element6],axis=1)
    except:
        a_element6=pd.Series(['아예 검색 없음'],name=i)
        a02_brand2=pd.concat([a02_brand2,a_element6],axis=1)

In [61]:
a02_brand

,start,5,6,7,8,9,10,11,12,13,...,4830,4831,4832,4833,4834,4835,4836,4837,4838,4839
0,시작,헤지스키즈,실리트,센스맘,빈폴스포츠,LG전자,가이거,클라비스,클라비스,스킨알엑스,...,세컨스킨,아디다스,닥스_핸드백,스킨알엑스,마루엔개비,빈폴ACC,빈폴ACC,에이치플러스몰,타미힐피거 여성,SOUP
1,NaN,헤지스ACC,1300K,NaN,시드마켓,아토세이프,폴브리알 시계,루미,클라비스,네이처리퍼블릭,...,엠피지,NaN,질스튜어트_핸드백,롭스,한샘,빈폴스포츠,빈폴 레이디스,버버리,타미힐피거여성,후라밍고
2,NaN,모노블랙,바보사랑,NaN,나야홍,삼성전자,NaN,NaN,클라비스,더샘,...,톰앤래빗,NaN,헤지스 가방,네이처 리퍼블릭,아씨방,빈폴-골프,NaN,프라다,타미힐피거,쇼크업소버코리아
3,NaN,레노마키즈,아트박스,NaN,헤지스골프,삼성전자,NaN,NaN,CLOVIS,NaN,...,아디다스,NaN,바보사랑,눙크,이케아,헤지스골프,NaN,발렌시아가,타미힐피거,마쉬옐로우
4,NaN,엠제이 주니어,실리트_,NaN,아디다스,대우전자,NaN,NaN,토모톰스,NaN,...,씨,NaN,후추통,온더바디,지비엠,NaN,NaN,발렌티노,타미힐피거,에이치스타일
5,NaN,NaN,룸멜스바하,NaN,반에이크 티알엔티,밀레,NaN,NaN,루미,NaN,...,베스띠벨리,NaN,엘케이트,이니스프리,시니프,NaN,NaN,구찌,폴로,퍼스웰
6,NaN,NaN,NaN,NaN,와이드앵글,벨,NaN,NaN,몽클레어,NaN,...,베스띠벨리,NaN,빌리버스*,네이처리퍼블릭,더리체,NaN,NaN,생로랑,타미힐피거진,닉스
7,NaN,NaN,NaN,NaN,막스마라,에브리케어,NaN,NaN,NaN,NaN,...,파파야,NaN,빈폴ACC,해피바스,마켓리더,NaN,NaN,돌체앤가바나,타미힐피거 슈즈,에이치플러스몰
8,NaN,NaN,NaN,NaN,아이더,위니아,NaN,NaN,NaN,NaN,...,비꼴리끄,NaN,기타,엘지생활건강,라자가구,NaN,NaN,지방시,타미힐피거 남성,사토리
9,NaN,NaN,NaN,NaN,네파,하이마트,NaN,NaN,NaN,NaN,...,아디다스,NaN,웅가로_넥타이,푸드어홀릭,나무뜰,NaN,NaN,레스포색,NaN,모슬린


In [63]:
a02_brand2.to_csv('a02_brand_ing2.csv')

In [65]:
a02_brand_ing=pd.read_csv('./a02_brand_ing.csv',low_memory=False)

In [66]:
a02_brand.head()

,start,5,6,7,8,9,10,11,12,13,...,4830,4831,4832,4833,4834,4835,4836,4837,4838,4839
0,시작,헤지스키즈,실리트,센스맘,빈폴스포츠,LG전자,가이거,클라비스,클라비스,스킨알엑스,...,세컨스킨,아디다스,닥스_핸드백,스킨알엑스,마루엔개비,빈폴ACC,빈폴ACC,에이치플러스몰,타미힐피거 여성,SOUP
1,NaN,헤지스ACC,1300K,NaN,시드마켓,아토세이프,폴브리알 시계,루미,클라비스,네이처리퍼블릭,...,엠피지,NaN,질스튜어트_핸드백,롭스,한샘,빈폴스포츠,빈폴 레이디스,버버리,타미힐피거여성,후라밍고
2,NaN,모노블랙,바보사랑,NaN,나야홍,삼성전자,NaN,NaN,클라비스,더샘,...,톰앤래빗,NaN,헤지스 가방,네이처 리퍼블릭,아씨방,빈폴-골프,NaN,프라다,타미힐피거,쇼크업소버코리아
3,NaN,레노마키즈,아트박스,NaN,헤지스골프,삼성전자,NaN,NaN,CLOVIS,NaN,...,아디다스,NaN,바보사랑,눙크,이케아,헤지스골프,NaN,발렌시아가,타미힐피거,마쉬옐로우
4,NaN,엠제이 주니어,실리트_,NaN,아디다스,대우전자,NaN,NaN,토모톰스,NaN,...,씨,NaN,후추통,온더바디,지비엠,NaN,NaN,발렌티노,타미힐피거,에이치스타일


In [69]:
a02_kwd3=a02_kwd[:5]

In [71]:
a02_brand3=pd.Series([1],name='start')

In [72]:
for i in range(len(a02_kwd3)):
    
    driver.implicitly_wait(randint(2,6))
    driver.get('http://www.lotteimall.com/main/viewMain.lotte?dpml_no=1&tlog=00100_1')
    a_element1=driver.find_element_by_id('headerQuery')
    a_element1.click()
    a_element1.send_keys(a02_kwd3[i])
    a_element2=driver.find_element_by_id("btn_headerSearch")
    a_element2.click()
    try:
        if ('결과' in driver.find_element_by_class_name('center').text) == True:

            a_element3=driver.find_element_by_id("brndList")
            a_element4=a_element3.text.split('\n')
            test=[]
            for j in range(len(a_element4)):
                text = re.sub(r'\([^)]*\)', repl='', string=a_element4[j])
                text=text.rstrip()
                test.append(text)
            a_element5=pd.Series(test,name=i)
            a02_brand3=pd.concat([a02_brand3,a_element5],axis=1)
        else:
            a_element6=pd.Series(['검색이 되지 않는다'],name=i)
            a02_brand3=pd.concat([a02_brand3,a_element6],axis=1)
    except:
        a_element6=pd.Series(['아예 검색 없음'],name=i)
        a02_brand3=pd.concat([a02_brand3,a_element6],axis=1)

In [70]:
a02_kwd3

['로가디스', '샌들', '슬리퍼', '신일에어서큘레이터', '생수2리터']

In [76]:
a02_brand3.drop(['start'],axis=1,inplace=True)
a02_brand.drop(['start'],axis=1,inplace=True)
a02_brand2.drop(['start'],axis=1,inplace=True)

,0,1,2,3,4,5,6,7,8,9,...,1519,1520,1521,1522,1523,1524,1525,1526,1527,1528
0,머렐,이지바이오메드,르까프,머렐,윌슨,넛츠앤,라푸마,아디다스,검색이 되지 않는다,베리슈,...,오버톤,삼성전자,삼성전자,폴햄키즈,LG전자,풍년,바보사랑,코렐,가이거,오야니
1,머렐,솔가,기타브랜드,머렐,윌슨,아트박스,NaN,아디다스,NaN,NaN,...,오픈클로젯,삼성전자,삼성전자,폴햄,NaN,풍년,디제이아이,NaN,NaN,NaN
2,머렐,지앤씨,케이스위스,머렐,파파브로,후추통,NaN,아디다스 퍼포먼스,NaN,NaN,...,보스,삼성전자,LG전자,폴햄키즈,NaN,NaN,1300K,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,엘르이너웨어,텐바이텐,NaN,아디다스 퍼포먼스_,NaN,NaN,...,NaN,삼성전자 가전,지펠,폴햄,NaN,NaN,게이즈샵,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,알콩단잠,이딸라,NaN,NaN,NaN,NaN,...,NaN,지펠,삼성전자 가전,폴햄키즈,NaN,NaN,헬셀,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,빈폴골프,데팡스,NaN,NaN,NaN,NaN,...,NaN,NaN,삼성전자,폴햄*,NaN,NaN,유쾌한생각,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,에어워크,FINLANDIA,NaN,NaN,NaN,NaN,...,NaN,NaN,딤채,엠폴햄*,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,헤지스골프,우리존,NaN,NaN,NaN,NaN,...,NaN,NaN,대유위니아,게스키즈,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,타미힐피거,1300K,NaN,NaN,NaN,NaN,...,NaN,NaN,렌탈프랜드,밀크북,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,신명글로빅스,비오투름,NaN,NaN,NaN,NaN,...,NaN,NaN,위니아,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
a02_bra1=a02_brand.copy()
a02_bra2=a02_brand2.copy()
a02_bra3=a02_brand3.copy()

In [88]:
a02_bra1=a02_bra1.iloc[:5,:]
a02_bra2=a02_bra2.iloc[:5,:]
a02_bra3=a02_bra3.iloc[:5,:]

In [91]:
a02_bra=pd.concat([a02_bra3,a02_bra1,a02_bra2],axis=1)

In [92]:
a02_bra

,0,1,2,3,4,5,6,7,8,9,...,1519,1520,1521,1522,1523,1524,1525,1526,1527,1528
0,로가디스,아이슈즈,아이슈즈,신일,스파클,헤지스키즈,실리트,센스맘,빈폴스포츠,LG전자,...,오버톤,삼성전자,삼성전자,폴햄키즈,LG전자,풍년,바보사랑,코렐,가이거,오야니
1,로가디스 시계,고세,멜팅브라운,신일산업,한모금,헤지스ACC,1300K,NaN,시드마켓,아토세이프,...,오픈클로젯,삼성전자,삼성전자,폴햄,NaN,풍년,디제이아이,NaN,NaN,NaN
2,NaN,멜팅브라운,텐바이텐,신일산업,기타,모노블랙,바보사랑,NaN,나야홍,삼성전자,...,보스,삼성전자,LG전자,폴햄키즈,NaN,NaN,1300K,NaN,NaN,NaN
3,NaN,크록스,에이치플러스몰,신일산업,네슬레,레노마키즈,아트박스,NaN,헤지스골프,삼성전자,...,NaN,삼성전자 가전,지펠,폴햄,NaN,NaN,게이즈샵,NaN,NaN,NaN
4,NaN,소다,디자이어로즈,NaN,롯데칠성음료,엠제이 주니어,실리트_,NaN,아디다스,대우전자,...,NaN,지펠,삼성전자 가전,폴햄키즈,NaN,NaN,헬셀,NaN,NaN,NaN


In [93]:
a02_bra.columns=a02_kwd

In [98]:
a02_bra.T.to_csv('final_a02_brand2.csv')

In [100]:
a02_bra.T.to_csv('final_a02_brand2_excel2.csv',encoding='UTF-8-sig')

In [101]:
driver.quit()